In [ ]:
import numpy as np
import pandas as pd
import re
from collections import defaultdict

In [ ]:
#data import
sal=pd.read_csv('salary.csv')
sal.drop('Unnamed: 0',axis=1,inplace=True)

In [ ]:
sal

,Country,Country code,Job,Avg. salary,Min. salary,Max. salary,Cities,Variation,Male percent,Female percent,Skills in demand
0,INDIA,IN,Assistant_General_Manager_(AGM),"₹1,687,306",₹628k,₹3m.,"Mumbai, Maharashtra",+14%,89.2%,10.6%,"['Project Management', 'Leadership', 'Business..."
1,INDIA,IN,Assistant_General_Manager_(AGM),"₹1,687,306",₹628k,₹3m.,"New Delhi, Delhi",+11%,89.2%,10.6%,"['Project Management', 'Leadership', 'Business..."
2,INDIA,IN,Assistant_General_Manager_(AGM),"₹1,687,306",₹628k,₹3m.,"Bangalore, Karnataka",+7%,89.2%,10.6%,"['Project Management', 'Leadership', 'Business..."
3,INDIA,IN,Assistant_General_Manager_(AGM),"₹1,687,306",₹628k,₹3m.,"Pune, Maharashtra",+4%,89.2%,10.6%,"['Project Management', 'Leadership', 'Business..."
4,INDIA,IN,Assistant_General_Manager_(AGM),"₹1,687,306",₹628k,₹3m.,"Delhi, Delhi",-4%,89.2%,10.6%,"['Project Management', 'Leadership', 'Business..."
...,...,...,...,...,...,...,...,...,...,...,...
1600,NEW ZEALND,NZ,Cyber_Security_Analyst,"NZ$66,564",NZ$50k,NZ$134k.,NEW ZEALND,0,87.5%,12.5%,['Security Information and Event Management (S...
1601,NEW ZEALND,NZ,Application_Developer,"NZ$63,214",NZ$48k,NZ$82k.,NEW ZEALND,0,73.3%,26.7%,"['Microsoft SQL Server', 'Java', 'C# Programmi..."
1602,NEW ZEALND,NZ,Web_Designer_%26_Developer,"NZ$57,249",NZ$42k,NZ$81k.,NEW ZEALND,0,63%,37%,"['Graphic Design', 'User Experience (UX) Desig..."
1603,NEW ZEALND,NZ,ScrumMaster,"NZ$95,902",NZ$67k,NZ$120k.,NEW ZEALND,0,41.7%,58.3%,"['Agile Software Development', 'ScrumMaster', ..."


In [ ]:
#removing , from the String salary format
def rep(a):
  return a.replace(',','')
sal['Avg. salary']=sal['Avg. salary'].apply(rep)

In [ ]:
#getting currency notatio and making A column for currency notation which can be shown in dashboard
def get_currency_not(a):
  pat='[0-9]'
  temp=re.sub(pat,'',a)
  return temp
sal['Currency Notation']=sal['Avg. salary'].apply(get_currency_not)

In [ ]:
#Re ordering the currency notation column 
cur_not=sal['Currency Notation'].copy()
sal=sal.drop('Currency Notation',axis=1)
sal.insert(3,'Currency Notation',cur_not)

In [ ]:
#Country's currency to USD approximate and converting it
country_to_dollars={'UNITED STATES':1, 'INDIA':0.014, 'GERMANY':1.18, 'RUSSIA':0.013, 'UNITED KINGDOMS':1.28, 'CHINA':0.15, 'SWITZERLAND':1.10, 'SWEDEN':0.11, 'NEW ZEALND':0.67, 'JAPAN':0.0094, 'NETHERLANDS':1.18, 'DENMARK':0.16, 'CANADA':0.76, 'AUSTRALIA':0.73}
def converter(a):
  return country_to_dollars[a]
sal['convert']=sal['Country'].apply(converter)

In [ ]:
#cleaning of Max,min and avg salary from string format to float format so that it can be useful in dashboard
def convert(a):
  max_sal=a['Max. salary'].strip()[:-1]
  min_sal=a['Min. salary'].strip()
  avg_sal=a['Avg. salary'].strip()
  fre='\d+\.\d+'
  ire='\d+'
  max_km=""
  min_km=""
  mul={'k':1000,'m':1000000}
  c=a['Country']
  ls=[m for m in re.finditer(fre,avg_sal)]
  if len(ls)==0: 
    ls=[m for m in re.finditer(ire,avg_sal)]
  l,r=ls[0].span()
  avg_sal_f=float(avg_sal[l:r])
  ls=[m for m in re.finditer(fre,max_sal)]
  if len(list(ls))==0:
    ls=[m for m in re.finditer(ire,max_sal)]
  l,r=ls[0].span()
  try:
    max_sal_f=float(max_sal[l:r])*mul[max_sal[r]]
  except:
    max_sal_f=float(max_sal[l:r])
  ls=[m for m in re.finditer(fre,min_sal)]
  if len(list(ls))==0:
    ls=[m for m in re.finditer(ire,min_sal)]
  l,r=ls[0].span()
  try:
    min_sal_f=float(min_sal[l:r])*mul[min_sal[r]]
  except:
    min_sal_f=float(min_sal[l:r])
  a['Max. salary']=max_sal_f
  a['Min. salary']=min_sal_f
  a['Avg. salary']=avg_sal_f
  return a
sal=sal.apply(convert,axis=1)

In [ ]:
#Creating Columns with USD currency from convert column for generalization
sal['Avg. salary (USD)']=sal['Avg. salary']*sal['convert']
sal['Max. salary (USD)']=sal['Max. salary']*sal['convert']
sal['Min. salary (USD)']=sal['Min. salary']*sal['convert']

In [ ]:
#saving with modification
sal.to_csv('modified salary.csv')

In [ ]:
#new scraped data with previous modification and new column Cost Of Living
new_sal=pd.read_csv('new_salary.csv')

In [ ]:
#Cleaning of Cost of living column
def COL_clean(a):
  a=a.replace(',','')
  fre='\d+\.\d+'
  ire='\d+'
  ls=[m for m in re.finditer(fre,a)]
  if len(ls)==0: 
    ls=[m for m in re.finditer(ire,a)]
  l,r=ls[0].span()
  return float(a[l:r])
new_sal['Cost of living pm']=new_sal['Cost of living pm'].apply(COL_clean)

In [ ]:
#checking if extra columns exist,if yes then remove it
new_sal.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Country', 'Country code', 'Job',
       'Currency Notation', 'Avg. salary', 'Min. salary', 'Max. salary',
       'Cities', 'Variation', 'Male percent', 'Female percent',
       'Skills in demand', 'convert', 'Avg. salary (USD)', 'Max. salary (USD)',
       'Min. salary (USD)', 'Cost of living pm'],
      dtype='object')

In [ ]:
new_sal=new_sal.drop(['Unnamed: 0','Unnamed: 0.1'],axis=1)

In [ ]:
new_sal

,Country,Country code,Job,Currency Notation,Avg. salary,Min. salary,Max. salary,Cities,Variation,Male percent,Female percent,Skills in demand,convert,Avg. salary (USD),Max. salary (USD),Min. salary (USD),Cost of living pm
0,INDIA,IN,AGM,₹,1687306.0,628000.0,3000000.0,"Mumbai, Maharashtra",14.0,89.2,10.6,"['Project Management', 'Leadership', 'Business...",0.014,23622.284,42000.0,8792.0,28405.58
1,INDIA,IN,AGM,₹,1687306.0,628000.0,3000000.0,"New Delhi, Delhi",11.0,89.2,10.6,"['Project Management', 'Leadership', 'Business...",0.014,23622.284,42000.0,8792.0,27215.80
2,INDIA,IN,AGM,₹,1687306.0,628000.0,3000000.0,"Bangalore, Karnataka",7.0,89.2,10.6,"['Project Management', 'Leadership', 'Business...",0.014,23622.284,42000.0,8792.0,26220.74
3,INDIA,IN,AGM,₹,1687306.0,628000.0,3000000.0,"Pune, Maharashtra",4.0,89.2,10.6,"['Project Management', 'Leadership', 'Business...",0.014,23622.284,42000.0,8792.0,24736.68
4,INDIA,IN,AGM,₹,1687306.0,628000.0,3000000.0,"Delhi, Delhi",-4.0,89.2,10.6,"['Project Management', 'Leadership', 'Business...",0.014,23622.284,42000.0,8792.0,27215.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1600,NEW ZEALND,NZ,Cyber Security Analyst,NZ$,66564.0,50000.0,134000.0,NEW ZEALND,NaN,87.5,12.5,['Security Information and Event Management (S...,0.670,44597.880,89780.0,33500.0,0.00
1601,NEW ZEALND,NZ,Application Developer,NZ$,63214.0,48000.0,82000.0,NEW ZEALND,NaN,73.3,26.7,"['Microsoft SQL Server', 'Java', 'C# Programmi...",0.670,42353.380,54940.0,32160.0,0.00
1602,NEW ZEALND,NZ,Web Designer,NZ$,57249.0,42000.0,81000.0,NEW ZEALND,NaN,63.0,37.0,"['Graphic Design', 'User Experience (UX) Desig...",0.670,38356.830,54270.0,28140.0,0.00
1603,NEW ZEALND,NZ,ScrumMaster,NZ$,95902.0,67000.0,120000.0,NEW ZEALND,NaN,41.7,58.3,"['Agile Software Development', 'ScrumMaster', ...",0.670,64254.340,80400.0,44890.0,0.00


In [ ]:
#Last stage of modification and cleaning
new_sal.to_csv('salary_fin.csv')